## Documentación del Modelo 1: Red Neuronal Artificial.

### Descripción

Inicialmente se pensó por utilizar un Supporting Vector Machine para predecir. Sin embargo, tras haber hecho pruebas no tuvo mucha presición al momento de clasificar, por lo que se optó por una red neuronal. Con la Red Neuronal Artificial lo que se busca es que cuando un nuevo jugador entre al juego, se pueda estimadar cuál sería la intensidad de ejercicios para cada nivel de dificultad en la primera sesión de juego. Esta intensidad se irá re-ajustando con base en el desempeño que los usuarios tengan al contestar los grupos de ejercicios que se le presenten en las futuras sesiones de juego. Sin embargo, la funcionalidad recientemente descrita será destinada para el modelo 2. Este modelo se encargará solo de los nuevos jugadores.

### Librerías

* __NumPy__: Para operaciones algebráicas y arreglos multidimensionales
* __Pandas__: Para estructuras de datos fáciles de usar, con alto desempeño y varias utilidades de analítica de datos
* __Seaborn__: Para visualizaciones de datos estadísticos con interfaz profesional.
* __Scikit-Learn__: Para realizar preprocesamiento y partición de datos, utilizar SVM y validar la precisión de los modelos de ML.
* __Keras__: Para utilizar redes neuronales de una forma rápida, sin complicaciones.
* __Pickle__: Para serializar objetos. Se usará para guardar los datos que describen a la red neuronal

In [9]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn import preprocessing;
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
#Keras library
import keras
from keras.models import Sequential, load_model
from keras.layers import Dense
# Feature Scaling
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.externals import joblib 
#Serialization
import pickle;


### Estructura del dataset

Los datos ya han sido previamente limpiados y están listos para usar. Se cargarán un total de 15500 datos, provenientes de 5 colegios. Los datos están distribuídos de la siguiente forma:

* __Colegio Hartfort__: 2875 datos provenientes de 115 niños (Grados 1-5) (M,F)
* __Colegio La Enseñanza__: 2800 datos provenientes de 112 niños (Grados 1-5) (F)
* __Colegio del Sagrado Corazón__: 1650 datos provenientes de 66 niños (robots) (Grados 1-5) (M,F)
* __Colegio Marco Fidel Suarez__: 3525 datos provenientes de 141 niños (Grados 1,2,4,5) (M,F)
* __Colegio Marie Paussepin__: 4650 datos provenientes de 186 niños (Grados 1-5) (F)

Donde 
* __M__: Significa Masculino
* __F__: Significa Femenino

Los datos serán cargados de los diferentes archivos .csv, que ya han pasado por un proceso de transformación y limpieza, por lo que no se encontrarán datos incompletos o erróneos o basura. La estructura de las tablas será la siguiente:

* __Edad__: La edad del niño
* __Escuela__: El nombre del colegio del niño
* __Género__: Género del niño, donde 0 es hombre y 1 es mujer.
* __Grado__: El grado que cursa actualmente (1-5)
* __LoID__: ID del objetivo de aprendizaje (0 a 4) asociado al ejercicio que contestó
* __Problema__: El ejercicio en concreto que resolvió
* __Tiempo__: El tiempo que tardó en contestar dicho ejercicio
* __userID__: El ID único de cada jugador (Dado por firebase)
* __isCorrect__: Integer que determina si el ejercicio lo contestó correctamente. 0 es si se equivocó y 1 es si lo contestó bien. Cabe destacar que cuando el niño contesta "no sé" o se equivoca, ambos son categorizados como un 0 (se equivocó). 
* __answerChangedCount__: Es un contador de cuantas veces el niño cambió entre las opciones de respuesta antes de enviar la que él consideraba como correcta. El único lugar donde no está presente es en el colegio Hartfort, por lo que se optó por asignarlo como 1 para todos los niños de ese dataset. (Luego se justificará por qué).

Se procede entonces a leer los datasets respectivos

In [10]:
#Reading the data
hartfortDF = pd.read_csv('./datasets/firstModel/datos_hartfort.csv')
hartfortDF = hartfortDF.drop(labels=['Unnamed: 0'],axis=1)
ensenanzaDF = pd.read_csv('./datasets/firstModel/datos_la_ensenanza.csv');
ensenanzaDF = ensenanzaDF.drop(labels=['Unnamed: 0'],axis=1)
marcoDF = pd.read_csv('./datasets/firstModel/datos_marco_fidel.csv');
marcoDF = marcoDF.drop(labels=['Unnamed: 0'],axis=1)
sagradoDF = pd.read_csv('./datasets/firstModel/datos_sagrado.csv');
sagradoDF = sagradoDF.drop(labels=['Unnamed: 0'],axis=1)
marieDF = pd.read_csv('./datasets/firstModel/datos_marie_poussepin.csv');
marieDF = marieDF.drop(labels=['Unnamed: 0'],axis=1)
hartfortDf.head(5)

NameError: name 'hartfortDf' is not defined

Guardaremos ahora los dataframe de escuelas en 3 grupos principales: Público, Privado, General. Esto es para hacer un análisis de cada grupo y ver cómo se comportan nuestros datos

In [3]:
schoolsDF = []
publicSchools = []
privateSchools = []
schoolsDF.append(hartfortDF)
schoolsDF.append(ensenanzaDF)
schoolsDF.append(marcoDF)
schoolsDF.append(sagradoDF)
schoolsDF.append(marieDF)

publicSchools.append(marcoDF)
publicSchools.append(marieDF)
privateSchools.append(hartfortDF)
privateSchools.append(ensenanzaDF)
privateSchools.append(sagradoDF)

,edad,genero,grado,loID,tiempo,userID,isCorrect
0,11,0,5,0,3.846108,-LaQZtfxo0QA8Ij62dk1,1
1,10,0,5,0,2.995306,-LaQZurPwCF55Hiv18kH,1
2,11,0,5,0,4.395411,-LaQZtfxo0QA8Ij62dk1,1
3,10,0,5,0,2.422333,-LaQZurPwCF55Hiv18kH,1
4,11,0,5,0,5.348062,-LaQZtfxo0QA8Ij62dk1,1


Lo que se hará entonces es agrupar los datos de cada niño. No nos interesa tanto saber cómo le fue en un ejercicio en específico, sino cómo le fue con todos los ejercicios del objetivo de aprendizaje. Hay que tener en cuenta que lo que se busca acá es un modelo general, entonces no se puede ir a cada ejercicio en particular, sino que queda mejor analizar por grupos de objetivos de aprendizaje. Entonces vamos a agrupar los datos de la siguiente forma:

* __Grupo general__: Esto nos servirá para acceder a los datos más fácilmente, ya que el groupby permite acceder a los valores por indices, como si fuera una matríz. Para el fragmento de código de abajo, si ejecutamos generalGroupBy(data), lo que entrega es un dataframe y las columnas "loID" y "grado" serían los índices (i,j) que pueden ser accedidos solo con los valores de las posiciones (números). Si no se hiciera el groupby, debería necesariamente hacerse una consulta con expresión lambda para poder acceder a los datos.
* __Grupo general por media de tiempo__: Lo mismo que el anterior, solo que ahora en vez de acceder a todos los otros datos por los que NO se agrupó, solo vamos a consderar el tiempo. Esto lo que retorna es el tiempo PROMEDIO que demoraron todos los niños de los diferentes grados, en los diferentes objetivos de aprendizaje.
* __Grupo general por mediana de tiempo__: Misma lógica que el anterior, solo que ahora ya no es el PROMEDIO sino la MEDIA. Si se grafican los datos de los tiempos, se puede ver que la gráfica que resulta es una gráfica con sesgo. Por tanto, utilizar la media de los datos cuando estos presentan una distribución sesgada no es recomendable, ya resulta demasiado sensible a los datos aberrantes. Es mejor para casos así utilizar la mediana ( y es por esto que se utiliza esta función).

Finalmente, reset_index() sirve para que los índices repetidos por el groupby se omitan.

In [4]:
def generalGroupBy(data):
    return (data.groupby(['loID','grado']).mean())

def generalGroupByTimeMEAN(data):
    return (data.groupby(['loID','grado'])['tiempo'].mean())

def generalGroupByTimeMEDIAN(data):
    return (data.groupby(['loID','grado'])['tiempo'].median())

###########################
####### Analysis ##########
schoolsMeansGB = []
schoolsMediansGB = []
schoolsGeneralGroupBy = []

schoolsMeansGB.append(generalGroupByTimeMEAN(hartfortDF).reset_index())
schoolsMeansGB.append(generalGroupByTimeMEAN(ensenanzaDF).reset_index())
schoolsMeansGB.append(generalGroupByTimeMEAN(marcoDF).reset_index())
schoolsMeansGB.append(generalGroupByTimeMEAN(sagradoDF).reset_index())
schoolsMeansGB.append(generalGroupByTimeMEAN(marieDF).reset_index())

schoolsMediansGB.append(generalGroupByTimeMEDIAN(hartfortDF).reset_index())
schoolsMediansGB.append(generalGroupByTimeMEDIAN(ensenanzaDF).reset_index())
schoolsMediansGB.append(generalGroupByTimeMEDIAN(marcoDF).reset_index())
schoolsMediansGB.append(generalGroupByTimeMEDIAN(sagradoDF).reset_index())
schoolsMediansGB.append(generalGroupByTimeMEDIAN(marieDF).reset_index())

schoolsGeneralGroupBy.append(generalGroupBy(hartfortDF).reset_index())
schoolsGeneralGroupBy.append(generalGroupBy(ensenanzaDF).reset_index())
schoolsGeneralGroupBy.append(generalGroupBy(marcoDF).reset_index())
schoolsGeneralGroupBy.append(generalGroupBy(sagradoDF).reset_index())
schoolsGeneralGroupBy.append(generalGroupBy(marieDF).reset_index())

## Selección de variables ideales para el modelo

Ahora que ya está listo el dataset, se busca encontrar variables que permitan ayudar a predecir la intensidad inicial de los ejercicios que se le colocará a un niño cuando entra por primera vez. 

Las variables con las que se cuenta son: **Edad**, **Género**, **Grado**, **Tiempo**, **Colegio**, **isCorrect**, **answerChangedCount**.

Hay que tener en cuenta que lo que se busca es un modelo general. El objetivo es que el niño no se enfrente a algún pre-test o que conteste ejercicios para balancearlo por primera vez, sino lograr una estimación general dada la información de la que se dispone al momento de que ellos se unen al juego. Por tanto, no se puede incorporar al modelo las variables de titubeo (answerChangedCount), respuesta correcta ni tiempo.

Lo que se hará ahora es realizar un análisis entre colegios para determinar si se presentan diferencias entre los colegios públicos y privados (Cosa que uno directamente se inclina a pensar y dice que sí la hay). Se procede pues, a separar los datos agrupados por colegios públicos y privados.

In [5]:
publicMeans = []
privateMeans = []
publicGeneral = []
privateGeneral = []
publicMeans.append(schoolsMeansGB[2])
publicMeans.append(schoolsMeansGB[4])
privateMeans.append(schoolsMeansGB[0])
privateMeans.append(schoolsMeansGB[1])
privateMeans.append(schoolsMeansGB[3])

publicGeneral.append(schoolsGeneralGroupBy[2])
publicGeneral.append(schoolsGeneralGroupBy[4])
privateGeneral.append(schoolsGeneralGroupBy[0])
privateGeneral.append(schoolsGeneralGroupBy[1])
privateGeneral.append(schoolsGeneralGroupBy[3])

Ahora, lo que se hará es definir una función general que permita comparar entre colegios cómo le va a los niños de cada grado con respecto a otras variables como: Cantidad de veces que cambiaron entre opción de respuesta, cantidad de preguntas correctas y el tiempo (promedio) que tardaron en contestar los ejercicios de un objetivo de aprendizaje.

dataComparison espera los siguientes parámetros:
* __Schools__: Una lista de las escuelas.
* __data__: El dataframe que se utilizará para sacar los datos (Medias, medianas, o alguno general)
* __row__: Una de las variables (X) que se utilizará para comparar con respecto a otra (Y)
* __col__: La segunda variable (Y) que está siendo comparada por la primera(X).
* __figIndex__: El ID de la figura o plot a dibujar.
* __Titulo__: El título de la gráfica.

Entonces, lo que se está comparando en el siguiente bloque es:

* __El promedio de tiempo que tardan los niños de cada grado__ en contestar los ejercicios de cada nivel de dificultad (Públicos).
* __El promedio de preguntas correctas que contestan los niños de cada grado__ en cada nivel de dificultad (Públicos).
* __El promedio de tiempo que tardan los niños de cada grado__ en contestar los ejercicios de cada nivel de dificultad (Privados).
* __El promedio de preguntas correctas que contestan los niños de cada grado__ en cada nivel de dificultad (Privados).


* __El PROMEDIO de tiempo que tardan los niños de cada grado__ en contestar los ejercicios de cada nivel de dificultad (Todos los colegios).
* __La MEDIANA de tiempo que tardan los niños de cada grado__ en contestar los ejercicios de cada nivel de dificultad (Todos los colegios).
* __El promedio de preguntas correctas que contestan los niños de cada grado__ en cada nivel de dificultad (Todos los colegios).

In [6]:
def dataComparison(schools,data,row,col,figIndex,titulo):
    for i in range(1,6):
        plt.figure(figIndex)
        ax = plt.subplot(2,3,i)
        for j in range(0,len(schools)):
            plt.plot(data[j].loc[lambda x: x.loID == i-1][row],data[j].loc[lambda x: x.loID == i-1][col], label=schools[j]['escuela'][1])
        
        ax.set_xlabel('Grado', fontsize=12)
        if(col=='isCorrect'):
            plt.ylim(0,1)
            ax.set_ylabel('% de correctas', fontsize=13)
        elif(col=='answerChangedCount'):
            plt.plot([1,2,3,4,5],[1,1,1,1,1], label='Ideal')
            ax.set_ylabel('Promedio de selección', fontsize=13)
            plt.ylim(0,2)
        else:
            ax.set_ylabel('Tiempo', fontsize=13)
        if(i==5):
            plt.suptitle(titulo)
            plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
        plt.title("Objetivo de aprendizaje #"+str(i), fontsize=13)
        

dataComparison(publicSchools, publicMeans,   'grado','tiempo',   24, 'Promedio de tiempos por colegio público')
dataComparison(publicSchools, publicGeneral, 'grado','isCorrect',25, 'Porcentaje de respuestas correctas por colegio público')
dataComparison(privateSchools,privateMeans,  'grado','tiempo',   26, 'Promedio de tiempos por colegio privado')
dataComparison(privateSchools,privateGeneral,'grado','isCorrect',27, 'Porcentaje de respuestas correctas por colegio privado')

#Time analysis MEAN
dataComparison(schoolsDF,schoolsMeansGB, 'grado', 'tiempo', 1,'Promedio de tiempos por colegio ')
#Time analysis MEDIAN (most accurate)
dataComparison(schoolsDF,schoolsMediansGB, 'grado', 'tiempo', 2, 'Mediana de tiempos por colegio')
#is Correct % analysis
dataComparison(schoolsDF,schoolsGeneralGroupBy, 'grado', 'isCorrect', 3, 'Preguntas correctas por colegio')
#changed Answers analysis
#dataComparison(schoolsDF,schoolsGeneralGroupBy, 'grado', 'answerChangedCount', 4)

Este análisis no muestra una diferencia significativa entre los colegios. Sí, puede verse que en algunos casos los niños de un mismo grado demoran más en algunos colegios que en otros, pero para decir que el colegio es un factor diferencial se esperaría que todos datos de análisis de los colegios públicos estuvieran separados de todos los colegios privados. Es decir, si se hubiera obtenido que los tiempos de ambos colegios privados siempre fueron mucho menores que todos los colegios públicos para un mismo grado, fuese fácil determinar que dicha variable es útil para encontrar patrones de datos. 

El género no se está mostrando en este análisis, pero modificando un poco la estructura de las funciones y llamados puede incorporarse para ver que el resultado será el mismo (porque ya se analizó previamente): No se puede ver una diferencia significativa entre los tiempos de los niños y niñas.

De igual manera, se optó por utilizar estas variables a pesar de que los análsis indiquen que no aportarán mucho al modelo, pues se cuenta con ellas al momento de que un niño ingresa por primera vez y se tiene la esperanza de que a medida que el dataset crezca estas empiecen a cobrar más fuerza.

### Estableciendo medidas de rendimiento

Luego de preparar el dataset, se procederá a establecer medidas de rendimiento que ayudarán al modelo a predecir las intensidades de ejercicios para cada nivel de dificultad. Como se tienen 5 objetivos de aprendizaje, hay 5 niveles de intensidad asociados a cada uno. El mejor indicador de desempeño del jugador es el tiempo que tomó en realizar un ejercicio. Por tanto, se optó por crear una fórmula que traduzca su desempeño en un valor fijo para tener un indicio de qué tan bien está el niño. 

### Puntaje

Para empezar, se definirá un puntaje base que disminuirá a medida que el jugador tome más tiempo para contestar la pregunta. Sin embargo, el tiempo que tarde en contestar será comparado directamente con la **mediana** del tiempo que tarden los niños que pertenezcan al mismo grado y colegio del jugador. Nuestra función entonces estará dada de la siguiente forma:

\begin{equation*}
score = \Bigr(20 - \frac{tiempo}{3}\Bigl) * isCorrect
\end{equation*}

Siendo:
* __tiempo__: El tiempo que tardó el jugador en contestar la pregunta.
* __isCorrect__: Si la pregunta fue correcta (1) o incorrecta (0).

Para evitar un puntaje negativo, tal que no se penalice al jugador muy fuerte, lo que se busca es que si se equivoca o tarda demasiado en resolver el ejercicio no se le otorgue puntaje. La primera versión de esta fórmula tenía establecido que si el jugador tardaba 60 segundos o más en contestar, el puntaje sería 0 (por esto el tiempo/3, para que cuando dure 60 segundos los puntos que le resten sean acordes al puntaje base de 20). Pero para que sea adaptativo a cada colegio, lo que se hizo fue establecer como frontera la mediana de tiempo por curso. Ahora, el tiempo límite que tendrá un niño para contestar el ejercicio estará definido por lo que hayan demorado los niños que pertenezcan a su mismo curso tras enfrentarse a un ejercicio del mismo nivel de dificultad.

Por esto, el siguiente fragmento de código trabaja con las siguientes variables:
* __answer['tiempo']__: Es el tiempo que tardó el niño en contestar el ejercicio.
* __timesMedian[ answer['loID'] , answer['grado'] ]__: Es la mediana de tiempo para un objetivo de aprendizaje (nivel de dificultad) específico __loID__, de un grado cualquiera (1 a 5).
* __timeLimit__: Para no hacerlo muy estricto, se otorgó arbitrariamente 2 segundos extra al tiempo. Así, para casos donde un niño tenga máxmimo 3 segundos de tiempo límite para contestar un ejercicio, ahora tendrá 5. (Esto se hizo pensando en la primera interacción con el juego, mientras se acostumbran a la dinámica).

In [7]:
def giveScoreToAnswer(answer, timesMedian):
    time = answer['tiempo'];
    timeLimit = int(timesMedian[answer['loID'],answer['grado']]) + 2;
    if(time < timeLimit and answer['isCorrect'] == 1):
        return 20-(time/3);
    else:
        return 0;

#Adding score to each school
hartfortDF['score'] = hartfortDF.apply(lambda x: giveScoreToAnswer(x, generalGroupByTimeMEDIAN(hartfortDF)), axis=1)
ensenanzaDF['score'] = ensenanzaDF.apply(lambda x: giveScoreToAnswer(x, generalGroupByTimeMEDIAN(ensenanzaDF)), axis=1)
marcoDF['score'] = marcoDF.apply(lambda x: giveScoreToAnswer(x, generalGroupByTimeMEDIAN(marcoDF)), axis=1)
sagradoDF['score'] = sagradoDF.apply(lambda x: giveScoreToAnswer(x, generalGroupByTimeMEDIAN(sagradoDF)), axis=1)
marieDF['score'] = marieDF.apply(lambda x: giveScoreToAnswer(x, generalGroupByTimeMEDIAN(marieDF)), axis=1)


Con esto, se tradujo el desempeño en cada ejercicio a un sistema de puntos. Saber cada ejercicio en específico no es el objetivo del modelo, así que se agrupará el desempeño del jugador por objetivos de aprendizaje, lo cual es posible por el ID de cada usuario.

In [8]:
scoresMeans = mergedDF.groupby(['userID','loID'])['score'].mean().reset_index();

,loID,grado,tiempo
0,0,1,11.064110
1,0,2,7.257710
2,0,3,6.910470
3,0,4,5.397869
4,0,5,3.801245
5,1,1,7.570393
6,1,2,4.913258
7,1,3,3.818320
8,1,4,2.991299
9,1,5,2.406552


Ahora mismo se tiene el promedio de puntos de cada niño en cada objetivo de aprendizaje. Trabajar el modelo para que prediga el puntaje que tendrá en cada objetivo según sus datos de proveniencia (Colegio, grado) y variables explicativas (género, edad) es una forma de atacar el problema. Sin embargo, en vez de medir qué tan bien le va, se optó por estimar qué tantos ejercicios debe colocarle.

Lo que se hace entonces es tranducir el puntaje de "qué tan bien está" en un "qué tantos ejercicios se le deben colocar" para dicho nivel de dificultad. Esto se logra de la siguiente forma:

### Intensidad de los LO

Lo que se hizo previamente fue tomar los puntajes obtenidos en los 5 ejercicios de cada objetivo de aprendizaje, sumarlos y dividirlos entre la cantidad de ejercicios. Partiendo de esto, si un niño contesta los 5 ejercicios perfectamente, su puntaje será o estará cerca de 20 (Máximo de puntos). 

Tomando su promedio de puntos y dividiéndolo entre el máximo de puntos posibles para obtener qué tan bien va, se obtiene un valor que considerado como **performance** para dicho objetivo de aprendizaje. La idea ahora es determinar una **intensidad** de ejercicios a presentar con base en dicho performance.  

\begin{eqnarray*}
General Score = \Bigl(\frac{\sum_{i=1}^{n}(Score_i)}{n}\Bigr)
\\
Performance = \frac{General Score}{20}
\end{eqnarray*}

Donde
* __n__: El número de ejercicios que realizó un jugador en un objetivo específico.
* __Score__: Es el puntaje que obtuvo el jugador para ese ejercicio
* __Score General__: El promedio de puntos entre los ejercicios resueltos.
* __Performance__: Es qué tan bien le fue al jugador para en un objetivo específico. Está medido en escala de 0 a 1, siendo 0 muy mal y 1 muy bien.

### Ejemplo

Si un niño contesta de forma correcta y rápida 5 ejercicios de algún nivel de dificultad, se espera que obtenga los siguientes puntajes:

\begin{eqnarray*}
Puntajes = \left(20,20,20,20,20\right)
\end{eqnarray*}

Por tanto, su score general sería el siguiente:

\begin{eqnarray*}
General Score = \Bigl(\frac{20+20+20+20+20}{5}\Bigr)
\end{eqnarray*}

Y por consiguiente, su performance:

\begin{eqnarray*}
Performance = \frac{20}{20}
\end{eqnarray*}

De este niño se puede interpretar que su performance en ese objetivo de aprendizaje es perfecto (1) y que no es necesario colocarle ejercicios. Lo que se puede hacer entonces es definir otra fórmula que a partir del performance determine la cantidad de ejercicios que se debe colocar. Se aprovechará que el performance da como resultado un porcentaje para establecer la siguiente fórmula:

\begin{eqnarray*}
LO Intensity = 1-performance
\end{eqnarray*}

En síntesis, la intensidad se calcula como **lo que le falta al niño para llegar a un performance perfecto de 1** para un objetivo de aprendizaje cualquiera. Este cálculo parte del puntaje obtenido tras contestar un grupo de ejercicios correspondiente a cada objetivo de aprendizaje, y será utilizado fuertemente en el __modelo 2__. Por último, redondeamos este valor a 1 cifra significativa para que al momento de predecir no se tengan en cuenta tantos decimales y que el modelo empiece a llorar.

In [9]:
def calculateIntensities(userPerformance):
    return (1-userPerformance.loc['score']/20);

scoresMeans['intensity'] = scoresMeans.apply(calculateIntensities,axis=1);
scoresMeans = scoresMeans.round({'intensity':1})

,edad,genero,grado,loID,tiempo,userID,isCorrect,score
0,11,0,5,0,3.846108,-LaQZtfxo0QA8Ij62dk1,1,18.717964
1,10,0,5,0,2.995306,-LaQZurPwCF55Hiv18kH,1,19.001565
2,11,0,5,0,4.395411,-LaQZtfxo0QA8Ij62dk1,1,18.534863
3,10,0,5,0,2.422333,-LaQZurPwCF55Hiv18kH,1,19.192556
4,11,0,5,0,5.348062,-LaQZtfxo0QA8Ij62dk1,1,0.000000


### Preparación del dataset

Ya se tiene definidas las principales variables a trabajar, pero todavía hay que realizar algunos ajustes. Previamente se intentó traducir los colegios a dummy variables y enviar esto como dato de entrada, pero luego se optó por generalizar aún más ya que, a la larga, no sería bueno tener un arreglo muy absurdo de Dummy Variables de colegios si el dataset llegaba a incrementar. Lo que se hace es tomar la lista de colegios y categorizarlos entre públicos y privados, para así tener los datos partidos en 2 grandes grupos. Así, a la larga, servirá para en un futuro (si el dataset llega a crecer muchísimo) poder validar si efectivamente el colegio es o no una variable influyente en el modelo y poder dar una conclusión más precisa de si debe tenerse en cuenta o no.

In [10]:
def convertSchool(user):
    if(user['escuela']=='Hartford International School'):
        return 0
    elif(user['escuela'] == 'I.E.D. Marie Poussepin'):
        return 3
    elif(user['escuela'] == 'I.E.D Marco Fidel Suárez'):
        return 4
    elif(user['escuela'] == 'Colegio del Sagrado Corazón'):
        return 1
    else: 
        return 2
    
def isPublicOrPrivate(user):
    if(user['escuela'] <= 2):
        return 1
    else:
        return 0

#General user information
mergedDF['escuela'] = mergedDF.apply(convertSchool,axis=1)
mergedDF['escuela'] = mergedDF.apply(isPublicOrPrivate,axis=1)

,userID,loID,score,intensity
0,-LaQZtfxo0QA8Ij62dk1,0,11.329112,0.6
1,-LaQZtfxo0QA8Ij62dk1,1,15.341532,0.8
2,-LaQZtfxo0QA8Ij62dk1,2,15.325926,0.8
3,-LaQZtfxo0QA8Ij62dk1,3,18.892539,0.9
4,-LaQZtfxo0QA8Ij62dk1,4,3.845159,0.2
5,-LaQZurPwCF55Hiv18kH,0,11.480608,0.6
6,-LaQZurPwCF55Hiv18kH,1,15.371654,0.8
7,-LaQZurPwCF55Hiv18kH,2,19.137623,1.0
8,-LaQZurPwCF55Hiv18kH,3,18.635188,0.9
9,-LaQZurPwCF55Hiv18kH,4,7.230178,0.4


Habiendo dejado lista la selección de variables de input, ahora se configurarán las variables de output. 

Como los datos permiten obtener una intensidad de cada niño para cada nivel de dificultad, se contará con un total de 5 intensidades. Este vector de 1x5 será agregado al dataframe actual, con la finalidad de analizar qué tan buena fue la selección de variables que se realizó para este primer modelo. 

In [11]:
def getIntensityDF(intensityDF):
    LO0Array = []
    LO1Array = []
    LO2Array = []
    LO3Array = []
    LO4Array = []
    IDArray = []
    for i in range(0, len(scoresMeans), 5):
        total = intensityDF.loc[i].intensity+intensityDF.loc[i+1].intensity+intensityDF.loc[i+2].intensity+intensityDF.loc[i+3].intensity+intensityDF.loc[i+4].intensity        
        if(total == 0):
            total = 1
        IDArray.append(intensityDF.loc[i].userID)
        LO0Array.append(intensityDF.loc[i].intensity/total)
        LO1Array.append(intensityDF.loc[i+1].intensity/total)
        LO2Array.append(intensityDF.loc[i+2].intensity/total)
        LO3Array.append(intensityDF.loc[i+3].intensity/total)
        LO4Array.append(intensityDF.loc[i+4].intensity/total)
    d = {'userID':IDArray, 'LOI0': LO0Array,'LOI1': LO1Array,'LOI2': LO2Array,'LOI3': LO3Array,'LOI4': LO4Array}
    return pd.DataFrame(data=d)

generalUserInfo = mergedDF.groupby(['userID']).mean()
generalUserInfo = generalUserInfo.drop(labels=['loID','tiempo','isCorrect','score'],axis=1);

LOIntensitiesPerUser = getIntensityDF(scoresMeans);
LOIntensitiesPerUser = LOIntensitiesPerUser.set_index('userID');
LOIntensitiesPerUser = LOIntensitiesPerUser.round(2);
generalUserInfo[['LOIN0','LOIN1','LOIN2','LOIN3','LOIN4']] = LOIntensitiesPerUser[['LOI0','LOI1','LOI2','LOI3','LOI4']];

,edad,genero,grado,LOIN0,LOIN1,LOIN2,LOIN3,LOIN4
userID,,,,,,,,
-LaQZtfxo0QA8Ij62dk1,11,0,5,0.6,0.8,0.8,0.9,0.2
-LaQZurPwCF55Hiv18kH,10,0,5,0.6,0.8,1.0,0.9,0.4
-LaQZvlOUvQcKzVILey9,10,0,5,0.8,1.0,0.6,0.8,0.2
-LaQZxvCZUvf9J0938NI,10,0,5,0.9,1.0,0.8,0.9,0.5
-LaQ_08Mr96o00YYSj4_,10,1,5,0.6,0.8,0.6,0.6,0.2


Se procede entonces a realizar un análsis de correlación con el método de Pearson, para ver qué tanto describen unas variables a otras.

In [ ]:
#Correlation analysis
correlationData = generalUserInfo.corr(method='pearson')

Como era de esperarse, solo el grado y la edad tendrían una alta correlación entre sí. Los resultados obtenidos por el método van acorde a los análisis de gráficas realizados previamente, donde se pudo ver que las demás variables no tenían mucha influencia en el desempeño del jugador. Al final de todo se encuentra un apartado dedicado a las mejoras que se pensaron para abordar este problema.

## Construcción de la Red Neuronal

Luego de que se tiene el dataset listo, se procede a partir entre variables independientes e independientes. Este proceso es para pasar los datos a la red neuronal.


### Arquitectura

El primer modelo de red neuronal que se diseñará se encargará de nivelar a un nuevo jugador. Cuando un nuevo jugador ingrese, la red deberá estimar las intensidades de los niveles de aprendizaje con base en el **colegio** (público o privado), **edad**, **género** y **grado**.

<img height=600, width=600, src="NeuralNetArch.png">

In [12]:
#Splitting the dataset
#Let X be Edad,Grado and Genero
X = generalUserInfo.drop(labels=['LOIN0','LOIN1','LOIN2','LOIN3','LOIN4'],axis=1);
#Let Y be the 5 intensities
Y = generalUserInfo[['LOIN0','LOIN1','LOIN2','LOIN3','LOIN4']].copy()

Se utilizará el 80% de los datos para entrenamiento y 20% para probar. Además, se deben escalar los datos para evitar la dominancia entre variables.

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=101)
sc = StandardScaler() #Must be scaled to avoid variable domination

#pickle.dumps(X.describe(),'normalizingData')
#X = preprocessing.normalize(X,axis=1)
trainedScaler = sc.fit(X_train);
#joblib.dump(sc, 'scaler.joblib');

#Transforming dataset
X_train = trainedScaler.transform(X_train)
X_test = trainedScaler.transform(X_test)

C:\Users\Steven\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:617: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Steven\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\Steven\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  """


### Construcción de la red

Utilizando la librería de Keras, se definirá la red neuronal como una secuencia de capas. Siguiendo el gráfico explicado en la sección anterior, tendrémos 3 capas: 

#### Input Layer
Recibirá los datos de entrada: Tipo de colegio, Edad, Género y Grado. Por tanto, la dimensión de esta capa debe ser 4.

#### Hidden Layer
Recibirá los datos de entrada y realizará diferentes combinaciones entre ellos buscandor relaciones para obtener las intensidades. Se utilizará la funcion de activcación de _Rectifier Linear Unit(ReLU)_ porque tradicionalmente es la que mejor performance da y hace que la red sea más fácil y rápido de entrenar a la red neuronal. Entre otras ventajas que presenta sobre otras funciones (sigmoide y tanh) es que esta no es tan sensible a los cambios. La cantidad de nodos que se llevará esta red es de 5. Como no hay una respuesta precisa de cuántas unidades se deben usar, se probaron diferentes cantidades de neuronas (3 a 10, 15, 20) se eligió la configuración que mejores resultados dió.

#### Output Layer
Contará con los 5 outputs respectivos a la intensidad de cada nivel de dificultad.

#### Kernel Initializers
Son las distribuciones con las que se asignarán los pesos a las aristas de la red neuronal. Entre distribución uniforme y normal no hay mucha diferencia en cuanto a qué tanto influyen en el modelo, pues ambos dan buenos resultados. Tomamos entonces una inicialización normal para las aristas de la hidden layer y una uniforme para la output layer.

#### Optimizer
Es la función para encontrar los valores óptimos. Como estamos haciendo una regresión, utilizamos el Stochastic Gradient Descent (SGD) ya que es mejor para aproximaciones.

#### Loss
La función de optimización de los pesos usaremos el error cuadrático medio (MSE). Esto principalmente por ser una regresión y porque se buscan valores que sean lo más cercanos posible.

#### Metrics
Para medir la precisión del modelo se utilziarán el MSE y el error absoluto medio (MAE).

### Entrenamiento de la red

#### batch_size
La cantidad de muestras que tomará la red para entrenarse al momento de hacer una propagación (paso entre input - hidden - output) para luego re calcular los pesos y ajustarse. Lo ideal es que no sea muy pequeño para que el movimiento del gradiente pueda dar resultados más precisos, pero tampoco muy grande para que no consuma mucha memoria. Se realizaron varias pruebas con cantidades variables de batch, pero no se obtuvieron resultados significativos cuando se optaba por usarlo o no. La versión final de la red trabaja sin batch pero, si se busca mejorar la arquitectura, no está de más probar con diferentes tamaños de batch para tratar de obtener mejores resultados.

#### epochs
La cantidad de veces que repetirá todo el proceso de entrenamiento con el dataset. 1 época se define como una pasada completa de todo el dataset, tras haber hecho proceso de forward y backward propagation.

Entonces para nuestra red que consta de 620 ejemplares para entrenar, analizarán todos de golpe para calcular los pesos. Al terminar, habrá finalizado una época. Entonces vuelve a repetirse 49 épocas más para mejorar su precisión.


In [14]:
#Building the ANN
classifier = Sequential();
classifier.add(Dense(input_dim=4, activation='relu', kernel_initializer='normal', units=5));
classifier.add(Dense(kernel_initializer='uniform',units=5))
classifier.compile(optimizer='sgd', loss='mean_squared_error', metrics=['mse','mae']);
modelHistory = classifier.fit(X_train, y_train, epochs=50)

y_pred = classifier.predict(X_test);

Epoch 1/50
92/92 [==============================] - 19s 204ms/step - loss: 0.2664 - mean_squared_error: 0.2664 - mean_absolute_error: 0.4224
Epoch 2/50
92/92 [==============================] - 0s 602us/step - loss: 0.2392 - mean_squared_error: 0.2392 - mean_absolute_error: 0.4000
Epoch 3/50
92/92 [==============================] - 0s 596us/step - loss: 0.2160 - mean_squared_error: 0.2160 - mean_absolute_error: 0.3792
Epoch 4/50
92/92 [==============================] - 0s 618us/step - loss: 0.1957 - mean_squared_error: 0.1957 - mean_absolute_error: 0.3598
Epoch 5/50
92/92 [==============================] - 0s 791us/step - loss: 0.1787 - mean_squared_error: 0.1787 - mean_absolute_error: 0.3422
Epoch 6/50
92/92 [==============================] - 0s 640us/step - loss: 0.1637 - mean_squared_error: 0.1637 - mean_absolute_error: 0.3257
Epoch 7/50
92/92 [==============================] - 0s 737us/step - loss: 0.1510 - mean_squared_error: 0.1510 - mean_absolute_error: 0.3128
Epoch 8/50
92/92 [=

Finalmente, esta configuración con la red neuronal que otorga un error de 0,0701 se guarda en un archivo. Este contendrá los pesos y toda la arquitectura que definimos para el funcionamiento de sus capas, tal que cuando se vuelva a utilizar solo sea cargar la información y no haya que volver a entrenar el modelo. De la misma forma, el StandardScaler utilizado para normalizar los datos de entrada fue guardado en un archivo .joblib para ser cargado.

Se finaliza entonces con una gráfica que contrasta los resultados obtenidos por la red neuronal con los datos reales. En ella se evidencia que el modelo realiza una regresión con base en la media, debido a que no cuenta con las suficientes variables explicativas para ajustarse a cada caso en particular.

In [15]:
def predictionsAnalysis(testValues, predictions, size):
    LO0 = []
    LO1 = []
    LO2 = []
    LO3 = []
    LO4 = []
    for i in range(0,size):
        LO0.append(predictions[i][0])
        LO1.append(predictions[i][1])
        LO2.append(predictions[i][2])
        LO3.append(predictions[i][3])
        LO4.append(predictions[i][4])
    
    for i in range(1,6):
        plt.figure(5)
        ax = plt.subplot(2,3,i)
        if(i == 1):
            testArray = testValues['LOIN0']
            LO = LO0;
        elif(i == 2):
            testArray = testValues['LOIN1']
            LO = LO1;
        elif(i == 3):
            testArray = testValues['LOIN2']
            LO = LO2;
        elif(i == 4):
            testArray = testValues['LOIN3']
            LO = LO3;
        else:
            testArray = testValues['LOIN4']
            LO = LO4;
            
        plt.scatter(list(range(0,size)),testArray)
        plt.scatter(list(range(0,size)),LO)
        plt.plot(list(range(0,size)),testArray,label='Valor Real')
        plt.plot(list(range(0,size)),LO,label='Predicción')
        ax.set_xlabel('Observación',fontsize=13)
        ax.set_ylabel('Intensidad',fontsize=13)
        plt.ylim(0,1)
        plt.title('Objetivo de aprendizaje #'+str(i))
        if(i == 5):
            plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
            plt.suptitle('Comparación entre valor real y la predicción del modelo')

predictionsAnalysis(y_test, y_pred, len(y_pred))
#classifier.save('NewPlayerNeuralNet.h5')

### Aspectos a mejorar

Entre las posibles alternativas que se pensaron para mejorar el modelo están las siguientes:
* __Inclusión de variables demográficas__: Posiblemente, contar con más variables que permitan tener información previa del niño permita encontrar algo que justifique su desempeño en cada nivel de dificultad.
* __Incluir información académica__: Posiblemente, incorporar otros datos como sus notas y rendimiento en el colegio permita que el modelo justifique su desempeño en cada nivel de dificultad.
* __Recolectar más datos provenientes de más colegios__: Es evidente que entre más datos hayan mejor se podrá entrenar cualquier modelo y puede que más particularidades puedan encontrarse. Siempre es recomendable levantar más datos, sobretodo porque para este proyecto no se contaba con algo como datos abiertos ni nada, sino que el dataset debió de crearse y a medida que se visitaban otros colegios se iban incorporando unas que otras variables.
* __Replantear la estrucutra del dataset__: Va de la mano con las 2 primeras sugerencias. Posiblemente si piensan en otra estructura para levantamiento de datos y así estudiarlos puede que esto conlleve a que se pueda formular un modelo más poderoso y preciso.
* __Aplicar las sugerencias de arquitectura__: Diferentes pruebas con otros optimizadores, tamaños de batch, epochs, inicializadores de pesos para las neuronas y diferentes random state de inicio